In [1]:
spark

In [2]:
# dir0 = '/home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/'
dir0 = '/home/master/iii_projects_data/kkbox_churn/data01/from_raw_transactions_v1/feature_creation/'

In [4]:
# subdir = '32.features_categorical/'
# items = ['actual_paid', 'auto_renew', 'discount', 'list_price', 'pay_method', 'plan_days']

subdir = '33.features_continuous/'
items = ['actual_paid', 'discount', 'list_price']

dfs = []
for it in items:
    infile = dir0+subdir+it
    df = spark.read.format('csv').option('header', 'true').load(infile)
    dfs.append(df)
    
df_join = dfs[0]
for df in dfs[1:]:
    df1 = df.withColumnRenamed('msno', 'msno_')
    df_join = df_join.join(df1, df_join['msno']==df1['msno_'], 'inner').drop('msno_')
    
outfile = dir0+subdir+'all-features'
df_join.write.format('csv').option('header','true').save(outfile)

In [5]:
infile = dir0+'30.features_gaps-related'
df30 = spark.read.format('csv').option('header','true').load(infile) \
       .drop('gaps')

In [6]:
subdirs = ['31.features_latest/', '32.features_categorical/', '33.features_continuous/']

df_join = df30
for subdir in subdirs:
    infile = dir0+subdir
    if not subdir.startswith('31.'):
        infile += '/'+'all-features'
    df = spark.read.format('csv').option('header','true').load(infile) \
         .withColumnRenamed('msno', 'msno_')
    df_join = df_join.join(df, df_join['msno']==df['msno_'], 'inner').drop('msno_')
    
outfile = dir0 + '34.all_trans_features/'
df_join.write.format('csv').option('header','true').save(outfile)